In [41]:
import os
import cv2
import time
import imutils
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from glob import glob
from tqdm import tqdm#_notebook as tqdm
from pyzbar import pyzbar

In [3]:
def rotate_bound(image, angle, borderValue):
    # grab the dimensions of the image and then determine the
    # center
    (h, w) = image.shape[:2]
    (cX, cY) = (w / 2, h / 2)

    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((cX, cY), angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])

    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))

    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY

    # perform the actual rotation and return the image
    return cv2.warpAffine(image, M, (nW, nH), borderValue=(borderValue,borderValue,borderValue))

In [4]:
y_filter = np.ones((9,9))
y_filter[:4] = -1
y_filter[4] = 0

In [5]:
def sobel(img, dx, dy, ksize):
    sobelx64f = cv2.Sobel(img,cv2.CV_64F,dx,dy,ksize=ksize)
    abs_sobel64f = np.absolute(sobelx64f)
    return np.uint8(abs_sobel64f)

In [6]:
def get_consecutive_coords_above_threshold(sobel_xy, height, width, squ_size, threshold):
    consecutive = 0 
    consecutive_coords = []
    text_coords = []
    for j in range(height//squ_size):
        sy = j*squ_size
        ey = j*squ_size + squ_size
        for i in range(width//squ_size):
            sx = i*squ_size
            ex = i*squ_size + squ_size

            squ = sobel_xy[sy:ey,sx:ex,:]

            _mean = squ.mean()
            if _mean >= threshold:
                consecutive += 1
                consecutive_coords += [(sx,sy,ex,ey)]

            elif consecutive >= 0:
                if consecutive >= min_consecutive:
                    consecutive = 0
                    text_coords += consecutive_coords
                else:
                    consecutive = 0
                    consecutive_coords = []

        if consecutive >= min_consecutive:
            text_coords += consecutive_coords
        consecutive_coords = []
        consecutive = 0
    
    return text_coords

def plot_rectangles(img, rectangles, colors=False):
    if not colors:
        messy = img.copy()
        for (startX, startY, endX, endY) in rectangles:
            cv2.rectangle(messy, (startX, startY), (endX, endY), (0, 255, 0), 2)
    
    if colors:
        colors = [(255,0,0),(0,255,0),(0,0,255),(255,255,0),(255,0,255),(0,255,255),(255,255,255),(100,100,100)]
        messy = img.copy()
        color = 0
        for label in rectangles:
            for (startX, startY, endX, endY) in label:
                cv2.rectangle(messy, (startX, startY), (endX, endY), colors[color], 2)
            color += 1
            if color == len(colors):
                color = 0
    plt.figure()
    plt.imshow(messy)
    
def group_squares_into_labels(coords, max_dist_btw, min_squ_per_label):
    coords_stack = coords.copy()

    labels = []
    while len(coords_stack):
        squ1 = coords_stack[0]
        sx1, sy1, ex1, ey1 = squ1
        mx1 = (sx1+ex1)/2
        my1 = (sy1+ey1)/2

        new_label = []
        to_be_checked = [squ1]
        coords_stack.remove(squ1)

        while len(to_be_checked):
            squ1 = to_be_checked[0]
            sx1, sy1, ex1, ey1 = squ1
            mx1 = (sx1+ex1)/2
            my1 = (sy1+ey1)/2

            new_label += [squ1]
            to_be_checked.remove(squ1)

            coords_stack2 = coords_stack.copy()
            for i in range(len(coords_stack2)):
                squ2 = coords_stack2[i]
                sx2, sy2, ex2, ey2 = squ2
                mx2 = (sx2+ex2)/2
                my2 = (sy2+ey2)/2
                dist = np.sqrt((mx1-mx2)**2 + (my1-my2)**2)
                if dist <= max_dist_btw:
                    to_be_checked += [squ2]
                    coords_stack.remove(squ2)
        if len(new_label) >= min_squ_per_label:
            labels += [new_label]
            
    return labels

In [15]:
fp.split('Cropped')[1].split('.jpg')[0]

'002'

In [91]:

debug = False

folderGlob = '/home/rob/HWR/datasets/zooniverse/**/Cropped00[2,4].jpg'
fps = glob(folderGlob)
if debug == True:
    fps = np.random.permutation(fps)

labels_data = []
for fp in tqdm(fps):
    if debug == True:
        fp = '/home/rob/HWR/datasets/zooniverse/222344/Cropped002.jpg'
    new_img_size = (480, 640)
    squ_size = 7
    assert squ_size%2 == 1
    max_dist_btw = squ_size*2.4
    min_consecutive = 3
    min_squ_per_label = 40
    threshold = 10
    height, width = new_img_size

    orig = cv2.imread(fp)
    orig_size = orig.shape[:2]
    scale = np.array(orig_size) / np.array(new_img_size)
    img = cv2.resize(orig, (width, height))
    
    img = cv2.GaussianBlur(img, (11,11), 3)
    sobel_xy = sobel(img, 1, 1, 5)    
    
    if debug == True:
        plt.imshow(img)
        plt.figure()
        plt.imshow(sobel_xy)

    text_coords = get_consecutive_coords_above_threshold(sobel_xy, height, width, squ_size, threshold)
    if debug == True:
        plot_rectangles(sobel_xy, text_coords)

    labels = group_squares_into_labels(text_coords, max_dist_btw, min_squ_per_label)
    if debug == True:
        plot_rectangles(img, labels, colors=True)
    
    sobel_y = sobel(img, 0, 1, 3)
    sobel_y = cv2.filter2D(img, -1, y_filter)
    sobel_y = cv2.cvtColor(sobel_y, cv2.COLOR_BGR2GRAY)
    _,sobel_y = cv2.threshold(sobel_y,120,255,cv2.THRESH_BINARY)

    
    #sobel_y = sobel_y / 255
    max_angle = 45
    n_angles = 300
    pad = 5
    
    
    if debug == True:
        plt.figure()
        plt.imshow(img)
        plt.figure()
        plt.imshow(sobel_y)
    i = 0
    catalog_no = fp.split('/')[6]
    for label in labels:
        label_data = [catalog_no, i]
        
        label = np.array(label)
        sx = max(label.min(axis=0)[0]-pad,0)
        sy = max(label.min(axis=0)[1]-pad,0)
        ex = min(label.max(axis=0)[2]+pad,width)
        ey = min(label.max(axis=0)[3]+pad,height)
        
        label_img = sobel_y[sy:ey,sx:ex].copy()
        
        messy = cv2.cvtColor(label_img, cv2.COLOR_GRAY2BGR).copy()
        thetas = []
        thres = 150
        min_len = 80
        max_gap = 20
        lines = cv2.HoughLinesP(label_img,1,np.pi/180,thres,300,min_len, max_gap)
        while (thres > 0) & (lines is None) & (min_len > 0):
            min_len -= 10
            thres -= 10
            max_gap += 10
            lines = cv2.HoughLinesP(label_img,1,np.pi/180,thres,300,min_len, max_gap)
            
        if lines is None:
            lines = []
        
        for line in lines:
            x1,y1,x2,y2 = line[0]
            cv2.line(messy,(x1,y1),(x2,y2),(0,255,0),2)
            thetas += [np.arctan((y2-y1)/(x2-x1))]
        
        if len(lines):
            best_angle = np.mean(thetas)*180/np.pi
        else:
            best_angle = 0
        
        if debug == True:
            plt.figure()
            plt.imshow(messy)
            plt.title(best_angle)    

        if debug == True:
            sx = int(scale[1]*(sx))
            sy = int(scale[0]*(sy))
            ex = int(scale[1]*(ex))
            ey = int(scale[0]*(ey))
            dst = rotate_bound(orig[sy:ey,sx:ex].copy(), best_angle, borderValue=255)
            plt.figure()
            plt.imshow(dst)
            plt.title(str(best_angle))

        
            

        M = cv2.getRotationMatrix2D((0, 0), best_angle, 1.0)
        rotated_squs = [np.append(a[:2],[1]) for a in label]
        rotated_squs += [np.append(a[2:],[1]) for a in label]
        rotated_squs = np.dot(rotated_squs, M.T)
        sx = rotated_squs.min(axis=0)[0]
        sy = rotated_squs.min(axis=0)[1]
        ex = rotated_squs.max(axis=0)[0]
        ey = rotated_squs.max(axis=0)[1]
        
        sx = int(scale[1]*(sx))
        sy = int(scale[0]*(sy))
        ex = int(scale[1]*(ex))
        ey = int(scale[0]*(ey))
        
        M = cv2.getRotationMatrix2D((0, 0), -best_angle, 1.0)
        corners = [[sx,sy,1],[sx,ey,1],[ex,sy,1],[ex,ey,1]]
        rotated_corners = np.dot(corners, M.T)
        rsx = int(rotated_corners.min(axis=0)[0]) - pad
        rsy = int(rotated_corners.min(axis=0)[1]) - pad
        rex = int(rotated_corners.max(axis=0)[0]) + pad
        rey = int(rotated_corners.max(axis=0)[1]) + pad
        label_data += [rsx, rsy, rex, rey]
        

        x_offset = 0
        y_offset = 0
        if best_angle < 0:
            x_offset = np.abs((orig.shape[0])*np.sin(best_angle*np.pi/180))
        elif best_angle > 0: 
            y_offset = np.abs((orig.shape[1])*np.sin(best_angle*np.pi/180))

        dst = rotate_bound(orig, best_angle, borderValue=255)
            
        nsx = int(sx + x_offset - pad)
        nsy = int(sy + y_offset - pad)
        nex = int(ex + x_offset + pad)
        ney = int(ey + y_offset + pad)
        nsx = max(nsx, 0)
        nsy = max(nsy, 0)
        nex = min(nex, dst.shape[1])
        ney = min(ney, dst.shape[0])
        nsx, nex = min(nsx, nex), max(nsx, nex)
        nsy, ney = min(nsy, ney), max(nsy, ney)
        
        
        aspect_ratio = (nex-nsx)/(ney-nsy)
        if aspect_ratio > 10:
            print('AR too large')
            continue
        
        
        if debug == True:
            plt.figure()
            plt.imshow(dst[nsy:ney,nsx:nex])
        
            plt.figure()
            plt.imshow(rotate_bound(dst[nsy:ney,nsx:nex],-best_angle,borderValue=255))
            plt.title(rotate_bound(dst[nsy:ney,nsx:nex],-best_angle,borderValue=255).shape)
        
        new_fp = fp.replace('Cropped','Label').replace('.jpg','%s.jpg'%i)
        _type = np.nan
        label_data += [new_fp, best_angle]
        if len(pyzbar.decode(dst[nsy:ney,nsx:nex])) | len(pyzbar.decode(cv2.resize(dst[nsy:ney,nsx:nex],(250,100)))):
            _type = 'QR Code'
        label_data += [_type]
        if debug == False:
            pass
            #cv2.imwrite(new_fp, dst[nsy:ney,nsx:nex])
        labels_data += [label_data]
        i+=1
    if debug == True:
        break








  0%|          | 0/3833 [00:00<?, ?it/s]





  0%|          | 1/3833 [00:03<3:28:42,  3.27s/it]





  0%|          | 2/3833 [00:05<3:05:38,  2.91s/it]





  0%|          | 3/3833 [00:08<3:17:17,  3.09s/it]





  0%|          | 4/3833 [00:11<3:14:34,  3.05s/it]





  0%|          | 5/3833 [00:13<2:58:03,  2.79s/it]





  0%|          | 6/3833 [00:17<3:06:55,  2.93s/it]





  0%|          | 7/3833 [00:22<3:44:00,  3.51s/it]





  0%|          | 8/3833 [00:24<3:21:56,  3.17s/it]





  0%|          | 9/3833 [00:27<3:20:42,  3.15s/it]





  0%|          | 10/3833 [00:30<3:20:20,  3.14s/it]





  0%|          | 11/3833 [00:33<3:09:01,  2.97s/it]





  0%|          | 12/3833 [00:37<3:32:19,  3.33s/it]





  0%|          | 13/3833 [00:39<3:11:11,  3.00s/it]





  0%|          | 14/3833 [00:42<3:11:25,  3.01s/it]

AR too large








  0%|          | 15/3833 [00:45<3:14:01,  3.05s/it]





  0%|          | 16/3833 [00:49<3:29:51,  3.30s/it]





  0%|          | 17/3833 [00:53<3:31:07,  3.32s/it]





  0%|          | 18/3833 [00:56<3:23:58,  3.21s/it]





  0%|          | 19/3833 [00:57<2:43:33,  2.57s/it]





  1%|          | 20/3833 [01:00<2:54:31,  2.75s/it]





  1%|          | 21/3833 [01:02<2:49:14,  2.66s/it]





  1%|          | 22/3833 [01:05<2:54:52,  2.75s/it]





  1%|          | 23/3833 [01:08<3:02:42,  2.88s/it]





  1%|          | 24/3833 [01:11<2:58:25,  2.81s/it]





  1%|          | 25/3833 [01:14<2:52:40,  2.72s/it]





  1%|          | 26/3833 [01:16<2:46:36,  2.63s/it]





  1%|          | 27/3833 [01:20<3:16:48,  3.10s/it]





  1%|          | 28/3833 [01:23<3:11:45,  3.02s/it]





  1%|          | 29/3833 [01:25<3:00:48,  2.85s/it]





  1%|          | 30/3833 [01:31<3:47:10,  3.58s/it]





  1%|          | 31/3833 [01:34<3:38:43,  3.45s/it]





  1%|          | 32/3833 

  3%|▎         | 123/3833 [06:44<4:39:27,  4.52s/it]





  3%|▎         | 124/3833 [06:49<4:56:54,  4.80s/it]

AR too large


...

 95%|█████████▌| 3656/3833 [3:12:52<10:26,  3.54s/it]





 95%|█████████▌| 3657/3833 [3:12:55<09:43,  3.32s/it]





 95%|█████████▌| 3658/3833 [3:12:58<09:10,  3.14s/it]





 95%|█████████▌| 3659/3833 [3:13:00<08:26,  2.91s/it]





 95%|█████████▌| 3660/3833 [3:13:03<08:13,  2.85s/it]





 96%|█████████▌| 3661/3833 [3:13:05<07:34,  2.64s/it]





 96%|█████████▌| 3662/3833 [3:13:07<06:52,  2.41s/it]





 96%|█████████▌| 3663/3833 [3:13:08<05:40,  2.00s/it]





 96%|█████████▌| 3664/3833 [3:13:11<07:04,  2.51s/it]





 96%|█████████▌| 3665/3833 [3:13:14<07:00,  2.51s/it]





 96%|█████████▌| 3666/3833 [3:13:17<07:20,  2.64s/it]





 96%|█████████▌| 3667/3833 [3:13:20<07:23,  2.67s/it]





 96%|█████████▌| 3668/3833 [3:13:22<07:05,  2.58s/it]





 96%|█████████▌| 3669/3833 [3:13:24<06:46,  2.48s/it]





 96%|█████████▌| 3670/3833 [3:13:27<06:56,  2.56s/it]





 96%|█████████▌| 3671/3833 [3:13:31<07:43,  2.86s/it]





 96%|█████████▌| 3672/3833 [3:13:34<07:52,  2.94s/it]




AR too large








 96%|█████████▌| 3679/3833 [3:13:52<07:55,  3.09s/it]





 96%|█████████▌| 3680/3833 [3:13:56<08:23,  3.29s/it]





 96%|█████████▌| 3681/3833 [3:13:59<07:57,  3.14s/it]





 96%|█████████▌| 3682/3833 [3:14:09<13:25,  5.33s/it]





 96%|█████████▌| 3683/3833 [3:14:12<11:03,  4.42s/it]





 96%|█████████▌| 3684/3833 [3:14:16<10:57,  4.41s/it]





 96%|█████████▌| 3685/3833 [3:14:19<10:09,  4.12s/it]





 96%|█████████▌| 3686/3833 [3:14:22<08:46,  3.58s/it]





 96%|█████████▌| 3687/3833 [3:14:32<13:33,  5.57s/it]





 96%|█████████▌| 3688/3833 [3:14:34<11:01,  4.56s/it]





 96%|█████████▌| 3689/3833 [3:14:37<09:57,  4.15s/it]





 96%|█████████▋| 3690/3833 [3:14:42<10:18,  4.33s/it]





 96%|█████████▋| 3691/3833 [3:14:46<09:50,  4.16s/it]





 96%|█████████▋| 3692/3833 [3:14:49<08:57,  3.81s/it]





 96%|█████████▋| 3693/3833 [3:14:53<08:59,  3.86s/it]





 96%|█████████▋| 3694/3833 [3:14:55<08:06,  3.50s/it]





 96%|█████████▋| 3695/3833 [3:14:58<07:18,  3.18s/

AR too large








 98%|█████████▊| 3752/3833 [3:18:09<04:10,  3.10s/it]





 98%|█████████▊| 3753/3833 [3:18:11<03:42,  2.79s/it]





 98%|█████████▊| 3754/3833 [3:18:14<03:50,  2.92s/it]





 98%|█████████▊| 3755/3833 [3:18:16<03:32,  2.72s/it]





 98%|█████████▊| 3756/3833 [3:18:19<03:33,  2.77s/it]





 98%|█████████▊| 3757/3833 [3:18:25<04:32,  3.58s/it]





 98%|█████████▊| 3758/3833 [3:18:27<04:14,  3.39s/it]





 98%|█████████▊| 3759/3833 [3:18:29<03:34,  2.90s/it]





 98%|█████████▊| 3760/3833 [3:18:32<03:26,  2.83s/it]





 98%|█████████▊| 3761/3833 [3:18:35<03:20,  2.79s/it]





 98%|█████████▊| 3762/3833 [3:18:38<03:26,  2.90s/it]





 98%|█████████▊| 3763/3833 [3:18:40<03:12,  2.75s/it]





 98%|█████████▊| 3764/3833 [3:18:44<03:31,  3.06s/it]





 98%|█████████▊| 3765/3833 [3:18:48<03:55,  3.46s/it]





 98%|█████████▊| 3766/3833 [3:18:50<03:16,  2.93s/it]





 98%|█████████▊| 3767/3833 [3:18:54<03:34,  3.24s/it]





 98%|█████████▊| 3768/3833 [3:18:58<03:45,  3.47s/

In [90]:
dst.shape, nsy, ney, nsx, nex,sy,ey,sx,ex#plt.imshow(dst[nsy:ney,nsx:nex])

((3053, 3157, 3), 1349, 1349, 620, 1322, 1354, 1650, 625, 1317)

In [64]:
       
new_fp = fp.replace('Cropped','Label').replace('.jpg','%s.jpg'%i)
_type = np.nan
label_data += [new_fp, best_angle]
if len(pyzbar.decode(dst[nsy:ney,nsx:nex])) | len(pyzbar.decode(cv2.resize(dst[nsy:ney,nsx:nex],(250,100)))):
    _type = 'QR Code'
label_data += [_type]
if debug == False:
    pass
    #cv2.imwrite(new_fp, dst[nsy:ney,nsx:nex])
labels_data += [label_data]
i+=1

ZeroDivisionError: integer division or modulo by zero

In [92]:
import pandas as pd
df = pd.DataFrame(labels_data, columns=['catalog_no','id','sx','sy','ex','ey','orig_fp','angle', 'type'])
df.to_csv('box coords.csv')

In [93]:
df

,catalog_no,id,sx,sy,ex,ey,orig_fp,angle,type
0,221812,0,210,285,1261,713,/home/rob/HWR/datasets/zooniverse/221812/Label...,0.077486,NaN
1,221812,1,2289,626,2885,1124,/home/rob/HWR/datasets/zooniverse/221812/Label...,4.344374,NaN
2,221812,2,446,822,1948,1214,/home/rob/HWR/datasets/zooniverse/221812/Label...,2.547739,NaN
3,221812,3,2255,1159,3201,1656,/home/rob/HWR/datasets/zooniverse/221812/Label...,-5.173400,NaN
4,221812,4,962,1868,2552,2425,/home/rob/HWR/datasets/zooniverse/221812/Label...,0.105055,QR Code
5,221223,0,1712,1212,2669,1789,/home/rob/HWR/datasets/zooniverse/221223/Label...,-8.947473,NaN
6,221223,1,612,1233,1277,1644,/home/rob/HWR/datasets/zooniverse/221223/Label...,-2.882963,NaN
7,221223,2,677,2039,2251,2635,/home/rob/HWR/datasets/zooniverse/221223/Label...,-0.988999,QR Code
8,222460,0,1687,255,2662,675,/home/rob/HWR/datasets/zooniverse/222460/Label...,1.912178,NaN
9,222460,1,1875,843,3060,1357,/home/rob/HWR/datasets/zooniverse/222460/Label...,-2.965034,NaN


In [47]:
label.shape

(735, 4)